# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [563]:
problem = '''A buyer needs to acquire 239,600,480 units of a product and is considering bids from five suppliers, labeled A through E, each of whom can only supply a portion of the total required amount.
Each vendor has proposed different pricing structures, incorporating both setup fees and variable unit costs that change based on the quantity ordered.

The buyer's objective is to allocate the order among these suppliers to minimize overall costs, accounting for both setup and unit costs.

Vendor A offers a set up cost of $3855.34 and a unit cost of $61.150 per thousand of units.
Vendor A can supply up to 33 million units.

Vendor B offers a set up cost of $125,804.84 if purchasing between 22,000,000-70,000,000 units from vendor B with a unit cost of $68.099 per thousand units.
If purchasing between 70,000,001-100,000,000 units from vendor B, the set up cost increases to $269304.84 and the unit cost sinks to $66.049 per thousand units.
If purchasing between 100,000,001-150,000,000 units from vendor B, the unit cost per thousand units further decreases to $64.099, but the set up cost increases to $464304.84.
If purchasing between 150,000,001 and 160,000,000 units from vendor B, the unit cost is $62.119 per thousand units and the set up cost equals $761304.84.

Vendor C offers set up costs of $13,456.00 and a unit cost of $62.019 per thousand units.
Vendor C can supply up to 165.6 million units. Vendor D offers set up costs of $6,583.98 and a unit cost of $72.488 for a set of thousand units.

Vendor D can supply up to 12 million units at a price of $72.488 per thousand units and with a set up cost of $6583.98.

Vendor E offers free set up if purchasing between 0 and 42 million units of vendor E with a unit price of $70.150 per thousand units.
If purchasing between 42,000,001 and 77 million units from vendor E, the unit cost starts at $68.150 per thousand units, but with every one million units purchased the price decreases at a rate of 0.05 percent. An additional set up cost of $84000 will be charged as well.

Note that zero units may be purchased from vendor B: otherwise no positive number of units less than 22,000,000 may be purchased.'''

## 2. Generate the mathematical model

In [572]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [573]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model

**Sets and Indices:**

*  *i* ∈ I = {A, B, C, D, E}: Set of vendors.
*  *j* ∈ J = {1, 2, 3, 4}: Set of price brackets for vendor B.
*  *k* ∈ K = {1, 2}: Set of price brackets for vendor E.

**Parameters:**

*  *D* = 239,600,480: Total units demanded.
*  *S<sub>i</sub>*: Setup cost for vendor *i*.
    *  *S<sub>A</sub>* = $3855.34
    *  *S<sub>B,1</sub>* = $125,804.84
    *  *S<sub>B,2</sub>* = $269,304.84
    *  *S<sub>B,3</sub>* = $464,304.84
    *  *S<sub>B,4</sub>* = $761,304.84
    *  *S<sub>C</sub>* = $13,456.00
    *  *S<sub>D</sub>* = $6,583.98
    *  *S<sub>E,1</sub>* = $0
    *  *S<sub>E,2</sub>* = $84,000
*  *C<sub>i</sub>*: Unit cost per thousand units for vendor *i*.
    *  *C<sub>A</sub>* = $61.150
    *  *C<sub>B,1</sub>* = $68.099
    *  *C<sub>B,2</sub>* = $66.049
    *  *C<sub>B,3</sub>* = $64.099
    *  *C<sub>B,4</sub>* = $62.119
    *  *C<sub>C</sub>* = $62.019
    *  *C<sub>D</sub>* = $72.488
    *  *C<sub>E,1</sub>* = $70.150
*  *L<sub>i</sub>*: Minimum quantity (in thousands) for vendor *i*.
    *  *L<sub>B</sub>* = 22,000
*  *U<sub>i</sub>*: Maximum quantity (in thousands) for vendor *i*.
    *  *U<sub>A</sub>* = 33,000
    *  *U<sub>B</sub>* = 160,000
    *  *U<sub>C</sub>* = 165,600
    *  *U<sub>D</sub>* = 12,000
    *  *U<sub>E</sub>* = 77,000
*  *B<sub>j</sub>*: Quantity breakpoints for vendor B (in thousands).
    *  *B<sub>1</sub>* = 22,000
    *  *B<sub>2</sub>* = 70,000
    *  *B<sub>3</sub>* = 100,000
    *  *B<sub>4</sub>* = 150,000
*  *E<sub>k</sub>*: Quantity breakpoints for vendor E (in thousands).
    *  *E<sub>1</sub>* = 0
    *  *E<sub>2</sub>* = 42,000

**Decision Variables:**

*  *x<sub>i</sub>*: Quantity (in thousands) purchased from vendor *i*.
*  *y<sub>i</sub>*: Binary variable, equal to 1 if any quantity is purchased from vendor *i*, 0 otherwise.
*  *z<sub>B,j</sub>*: Binary variable, equal to 1 if the quantity purchased from vendor B falls within price bracket *j*, 0 otherwise.
*  *w<sub>E,k</sub>*: Binary variable, equal to 1 if the quantity purchased from vendor E falls within price bracket *k*, 0 otherwise.

**Objective Function:**
Minimize total cost:
```
Minimize Σ<sub>i∈I</sub> (S<sub>i</sub> * y<sub>i</sub> + C<sub>i</sub> * x<sub>i</sub>) 
     + Σ<sub>j∈J</sub> (S<sub>B,j</sub> * z<sub>B,j</sub>) 
     + Σ<sub>k∈K</sub> (S<sub>E,k</sub> * w<sub>E,k</sub>)
```

**Constraints:**

1. **Demand Constraint:**
   ```
   Σ<sub>i∈I</sub> x<sub>i</sub> = D
   ```

2. **Minimum and Maximum Quantity Constraints:**
   ```
   L<sub>i</sub> * y<sub>i</sub> ≤ x<sub>i</sub> ≤ U<sub>i</sub> * y<sub>i</sub>, ∀ i ∈ I 
   ```

3. **Vendor B Price Bracket Constraints:**
   ```
   B<sub>j-1</sub> * z<sub>B,j</sub> < x<sub>B</sub> ≤ B<sub>j</sub> * z<sub>B,j</sub>, ∀ j ∈ J 
   Σ<sub>j∈J</sub> z<sub>B,j</sub> = y<sub>B</sub>
   ```

4. **Vendor E Price Bracket Constraints:**
   ```
   E<sub>k-1</sub> * w<sub>E,k</sub> < x<sub>E</sub> ≤ E<sub>k</sub> * w<sub>E,k</sub>, ∀ k ∈ K
   Σ<sub>k∈K</sub> w<sub>E,k</sub> = y<sub>E</sub> 
   ```

5. **Variable Type Constraints:**
   ```
   x<sub>i</sub> ≥ 0, ∀ i ∈ I
   y<sub>i</sub> ∈ {0, 1}, ∀ i ∈ I
   z<sub>B,j</sub> ∈ {0, 1}, ∀ j ∈ J
   w<sub>E,k</sub> ∈ {0, 1}, ∀ k ∈ K
   ```

**Note:** This model does not account for the decreasing unit cost within the second price bracket of vendor E. To accommodate this, a more complex model with piecewise linear functions would be needed.


## 3. Generate the pyomo code

In [574]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [575]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data
D = 239600.480  # Total units demanded (in thousands)

S = {"A": 3855.34, "C": 13456.00, "D": 6583.98}
S_B = {1: 125804.84, 2: 269304.84, 3: 464304.84, 4: 761304.84}
S_E = {1: 0, 2: 84000}

C = {"A": 61.150, "C": 62.019, "D": 72.488}
C_B = {1: 68.099, 2: 66.049, 3: 64.099, 4: 62.119}
C_E = {1: 70.150}  # Note: This does not account for decreasing cost within bracket

L = {"B": 22000}
U = {"A": 33000, "B": 160000, "C": 165600, "D": 12000, "E": 77000}

B = {1: 22000, 2: 70000, 3: 100000, 4: 150000}
E = {1: 0, 2: 42000}

# Set and Indices
I = ["A", "B", "C", "D", "E"] # vendors
J = [1, 2, 3, 4] # price brackets for vendor B
K = [1, 2] # price brackets for vendor E

# Model
model = pyo.ConcreteModel()

# Decision Variables
model.x = pyo.Var(I, within=pyo.NonNegativeReals) # quantity purchased from each vendor
model.y = pyo.Var(I, within=pyo.Binary) # 1 if any quantity is purchased from vendor i, 0 otherwise
model.z = pyo.Var(J, within=pyo.Binary) # 1 if purchase from vendor B falls in price bracket j
model.w = pyo.Var(K, within=pyo.Binary) # 1 if purchase from vendor E falls in price bracket k

# Objective Function
model.obj = pyo.Objective(expr=sum(S[i]*model.y[i] + C[i]*model.x[i] for i in I if i != 'B' and i != 'E') +
                         sum(S_B[j]*model.z[j] for j in J) + 
                         sum(S_E[k]*model.w[k] for k in K) +
                         sum(C_B[j]*model.x['B'] for j in J) +
                         C_E[1]*model.x['E'], 
                         sense=pyo.minimize) 

# Constraints
# Demand Constraint
model.demand_constraint = pyo.Constraint(expr=sum(model.x[i] for i in I) == D)

# Minimum and Maximum Quantity Constraints
model.min_max_constraints = pyo.ConstraintList()
for i in I:
    if i in L:
        model.min_max_constraints.add(expr=L[i]*model.y[i] <= model.x[i])
    if i in U:
        model.min_max_constraints.add(expr=model.x[i] <= U[i]*model.y[i])

# Vendor B Price Bracket Constraints
model.vendor_b_constraints = pyo.ConstraintList()
for j in J:
    if j > 1:
        model.vendor_b_constraints.add(expr=B[j-1]*model.z[j] <= model.x['B'])
    model.vendor_b_constraints.add(expr=model.x['B'] <= B[j]*model.z[j]) 
model.vendor_b_constraints.add(expr=sum(model.z[j] for j in J) == model.y['B'])

# Vendor E Price Bracket Constraints
model.vendor_e_constraints = pyo.ConstraintList()
for k in K:
    if k > 1:
        model.vendor_e_constraints.add(expr=E[k-1]*model.w[k] <= model.x['E'])
    model.vendor_e_constraints.add(expr=model.x['E'] <= E[k]*model.w[k]) 
model.vendor_e_constraints.add(expr=sum(model.w[k] for k in K) == model.y['E'])

# Solve the model
solver = pyo.SolverFactory('cbc') # Use CBC solver, you can change to other solvers
results = solver.solve(model)

# Print the results
print(results)
print("Optimal Solution:")
for i in I:
    print(f"x[{i}] = {model.x[i].value}")
```

**Notes:**

*   The code uses the sample data you provided in the problem statement.
*   This Pyomo code represents the mathematical optimization model you described, including the objective function and all the constraints.
*   This model utilizes the CBC solver. You can explore different solvers by changing the `SolverFactory` argument.
*   The model does not include the decreasing unit cost within Vendor E's second price bracket. To incorporate this, you would need to introduce piecewise linear functions within the model.


## 4. Input problem data and try running the generated code

In [577]:
import pyomo.environ as pyo

# Sample Data
D = 239600.480  # Total units demanded (in thousands)

S = {"A": 3855.34, "C": 13456.00, "D": 6583.98}
S_B = {1: 125804.84, 2: 269304.84, 3: 464304.84, 4: 761304.84}
S_E = {1: 0, 2: 84000}

C = {"A": 61.150, "C": 62.019, "D": 72.488}
C_B = {1: 68.099, 2: 66.049, 3: 64.099, 4: 62.119}
C_E = {1: 70.150}  # Note: This does not account for decreasing cost within bracket

L = {"B": 22000}
U = {"A": 33000, "B": 160000, "C": 165600, "D": 12000, "E": 77000}

B = {1: 22000, 2: 70000, 3: 100000, 4: 150000}
E = {1: 0, 2: 42000}

# Set and Indices
I = ["A", "B", "C", "D", "E"] # vendors
J = [1, 2, 3, 4] # price brackets for vendor B
K = [1, 2] # price brackets for vendor E

# Model
model = pyo.ConcreteModel()

# Decision Variables
model.x = pyo.Var(I, within=pyo.NonNegativeReals) # quantity purchased from each vendor
model.y = pyo.Var(I, within=pyo.Binary) # 1 if any quantity is purchased from vendor i, 0 otherwise
model.z = pyo.Var(J, within=pyo.Binary) # 1 if purchase from vendor B falls in price bracket j
model.w = pyo.Var(K, within=pyo.Binary) # 1 if purchase from vendor E falls in price bracket k

# Objective Function
model.obj = pyo.Objective(expr=sum(S[i]*model.y[i] + C[i]*model.x[i] for i in I if i != 'B' and i != 'E') +
                         sum(S_B[j]*model.z[j] for j in J) + 
                         sum(S_E[k]*model.w[k] for k in K) +
                         sum(C_B[j]*model.x['B'] for j in J) +
                         C_E[1]*model.x['E'], 
                         sense=pyo.minimize) 

# Constraints
# Demand Constraint
model.demand_constraint = pyo.Constraint(expr=sum(model.x[i] for i in I) == D)

# Minimum and Maximum Quantity Constraints
model.min_max_constraints = pyo.ConstraintList()
for i in I:
    if i in L:
        model.min_max_constraints.add(expr=L[i]*model.y[i] <= model.x[i])
    if i in U:
        model.min_max_constraints.add(expr=model.x[i] <= U[i]*model.y[i])

# Vendor B Price Bracket Constraints
model.vendor_b_constraints = pyo.ConstraintList()
for j in J:
    if j > 1:
        model.vendor_b_constraints.add(expr=B[j-1]*model.z[j] <= model.x['B'])
    model.vendor_b_constraints.add(expr=model.x['B'] <= B[j]*model.z[j]) 
model.vendor_b_constraints.add(expr=sum(model.z[j] for j in J) == model.y['B'])

# Vendor E Price Bracket Constraints
model.vendor_e_constraints = pyo.ConstraintList()
for k in K:
    if k > 1:
        model.vendor_e_constraints.add(expr=E[k-1]*model.w[k] <= model.x['E'])
    model.vendor_e_constraints.add(expr=model.x['E'] <= E[k]*model.w[k]) 
model.vendor_e_constraints.add(expr=sum(model.w[k] for k in K) == model.y['E'])

# Solve the model
solver = pyo.SolverFactory('glpk') # Use CBC solver, you can change to other solvers
results = solver.solve(model)

# Print the results
print(results)
print("Optimal Solution:")
for i in I:
    print(f"x[{i}] = {model.x[i].value}")


Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 19
  Number of variables: 16
  Number of nonzeros: 43
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: infeasible
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0390925407409668

Optimal Solution:
x[A] = None
x[B] = None
x[C] = None
x[D] = None
x[E] = None


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [578]:
print(response.text)

## Mathematical Optimization Model

**Sets and Indices:**

*  *i* ∈ I = {A, B, C, D, E}: Set of vendors.
*  *j* ∈ J = {1, 2, 3, 4}: Set of price brackets for vendor B.
*  *k* ∈ K = {1, 2}: Set of price brackets for vendor E.

**Parameters:**

*  *D* = 239,600,480: Total units demanded.
*  *S<sub>i</sub>*: Setup cost for vendor *i*.
    *  *S<sub>A</sub>* = $3855.34
    *  *S<sub>B,1</sub>* = $125,804.84
    *  *S<sub>B,2</sub>* = $269,304.84
    *  *S<sub>B,3</sub>* = $464,304.84
    *  *S<sub>B,4</sub>* = $761,304.84
    *  *S<sub>C</sub>* = $13,456.00
    *  *S<sub>D</sub>* = $6,583.98
    *  *S<sub>E,1</sub>* = $0
    *  *S<sub>E,2</sub>* = $84,000
*  *C<sub>i</sub>*: Unit cost per thousand units for vendor *i*.
    *  *C<sub>A</sub>* = $61.150
    *  *C<sub>B,1</sub>* = $68.099
    *  *C<sub>B,2</sub>* = $66.049
    *  *C<sub>B,3</sub>* = $64.099
    *  *C<sub>B,4</sub>* = $62.119
    *  *C<sub>C</sub>* = $62.019
    *  *C<sub>D</sub>* = $72.488
    *  *C<sub>E,1</sub>* = $70.150
*

In [580]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data
D = 239600.480  # Total units demanded (in thousands)

S = {"A": 3855.34, "C": 13456.00, "D": 6583.98}
S_B = {1: 125804.84, 2: 269304.84, 3: 464304.84, 4: 761304.84}
S_E = {1: 0, 2: 84000}

C = {"A": 61.150, "C": 62.019, "D": 72.488}
C_B = {1: 68.099, 2: 66.049, 3: 64.099, 4: 62.119}
C_E = {1: 70.150}  # Note: This does not account for decreasing cost within bracket

L = {"B": 22000}
U = {"A": 33000, "B": 160000, "C": 165600, "D": 12000, "E": 77000}

B = {1: 22000, 2: 70000, 3: 100000, 4: 150000}
E = {1: 0, 2: 42000}

# Set and Indices
I = ["A", "B", "C", "D", "E"] # vendors
J = [1, 2, 3, 4] # price brackets for vendor B
K = [1, 2] # price brackets for vendor E

# Model
model = pyo.ConcreteModel()

# Decision Variables
model.x = pyo.Var(I, within=pyo.NonNegativeReals) # quantity purchased from each vendor
model.y = pyo.Var(I, within=pyo.Binary) # 1 if any quantity is purchased from vendor i, 0 otherwise
model.z = pyo.Var(J, within=py